In [1]:
from gurobipy import GRB
import gurobipy as gb

In [2]:
model = gb.Model("ED Staffing")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-12


In [3]:
x = model.addVars(28, 3, 3, vtype = GRB.BINARY, name="Physician i, Staffed On Shift j, on Day k")

Import Data Respective to Each Physician 

In [4]:
Physician_speed = [
234.37092731829574,
259.7377165577961,
999.0,
262.5791726105563,
183.57051460361615,
217.92519685039372,
262.6868131868132,
268.8324258099646,
243.86320109439126,
269.22103592555186,
242.03161460293563,
250.64790973528133,
228.53105802047781,
211.19029374201787,
223.32002249718786,
280.048231511254,
249.26356835906415,
290.7238095238095,
268.7877358490566,
999.0,
226.9508990318119,
211.70398277717976,
256.45701096364684,
199.46953781512605,
200.699158174568,
223.15081339712918,
227.2958928571429,
220.85367454068242]

Physician_quality = [
0.04211162603264154,
0.028020623178659495,
999.0,
0.03969957081545064,
0.02779937791601866,
0.03539292141571686,
0.027027027027027032,
0.0344896731501905,
0.027281063732232918,
0.029454722492697177,
0.02750146284376829,
0.032562125107112254,
0.023575638506876228,
0.023508137432188065,
0.04063745019920319,
0.01762114537444934,
0.04724880382775119,
0.015873015873015872,
0.00684931506849315,
999.0,
0.036702127659574466,
0.04199475065616798,
0.02601156069364162,
0.02787286063569682,
0.040104053761109905,
0.03651354534746761,
0.03734827264239029,
0.03755868544600939]

Physician_capacity = [
362.71204188481676,
368.22222222222223,
0.0,
446.94827586206895,
292.0394088669951,
336.9267015706806,
446.9130434782609,
408.07894736842104,
385.3030303030303,
426.13775510204084,
390.84,
361.1803278688525,
350.42105263157896,
425.030303030303,
386.90714285714284,
592.421052631579,
404.3492063492063,
415.44444444444446,
426.0,
0.0,
358.90425531914894,
324.5769230769231,
363.4761904761905,
305.5584415584416,
365.3368983957219,
353.84276729559747,
332.0506329113924,
339.26190476190476]

2D array of E[X] value pairs

In [5]:
EX_values = [[2615.0100304588204, 2224.3150834052317, 2116.8572771478744],
 [933.7482651610213, 817.945374108041, 764.2008534653297],
 [293.94786258404037, 317.1954847371322, 339.20551283922543]]

Objective Function

In [6]:
model.setObjective(gb.quicksum(
    (x[i,j,k] * Physician_speed[i] + x[i,j,k] * Physician_quality[i]) * EX_values[j][k]
    for i in range(28) for j in range(3) for k in range(3)), GRB.MINIMIZE)

Contraints (Set 1)

Which Physicians Don't Exist 

In [7]:
for j in range(3): #Physician 3 not in dataset
    for k in range(3):
        model.addConstr(x[2,j,k]==0)

for j in range(3): #Physician 20 not in dataset
    for k in range(3):
        model.addConstr(x[19,j,k]==0)

Which Physicians No Longer Work in the Department based on the Last 6 Months

In [8]:
for j in range(3): #Physician 22
    for k in range(3):
        model.addConstr(x[21,j,k]==0)

for j in range(3): #Physician 28
    for k in range(3):
        model.addConstr(x[27,j,k]==0)

Demand Constraints

In [9]:
for j in range(3):
    for k in range(3):
        model.addConstr(gb.quicksum(x[i,j,k]*Physician_capacity[i] for i in range(28))>=EX_values[j][k])
        

Labour Constraints

Only One Shift Per Day

In [10]:
for i in range(28):
    for k in range(3):
        model.addConstr(gb.quicksum(x[i, j, k] for j in range(3)) <= 1)


Night Shift is Not Followed by Day Shift

In [11]:
for i in range(28):
    for k in range(2):
        model.addConstr(x[i, 2, k] + x[i, 0, k+1] <= 1)

Physician Specific Constraints (Set 2)

Will Not Work Certain Shifts

In [12]:
for k in range(3):
    model.addConstr(x[6, 1, k]==0)

for k in range(3):
    model.addConstr(x[6, 2, k]==0)

for k in range(3):
    model.addConstr(x[12, 2, k]==0)

for k in range(3):
    model.addConstr(x[15, 1, k]==0)

for k in range(3):
    model.addConstr(x[15, 2, k]==0)

for k in range(3):
    model.addConstr(x[17, 1, k]==0)

for k in range(3):
    model.addConstr(x[17, 2, k]==0)

for k in range(3):
    model.addConstr(x[18, 1, k]==0)

for k in range(3):
    model.addConstr(x[18, 2, k]==0)

for k in range(3):
    model.addConstr(x[20, 2, k]==0)

Will Not Work Certain Days

In [13]:
for j in range(3):
    model.addConstr(x[0, j, 0]==0)

for j in range(3):
    model.addConstr(x[0, j, 1]==0)

for j in range(3):
    model.addConstr(x[6, j, 0]==0)

Reserve Physicians at Most Once a Week (Set 3)

In [14]:
model.addConstr(gb.quicksum(x[6,j,k] for j in range(3) for k in range(3))<=1)
model.addConstr(gb.quicksum(x[12,j,k] for j in range(3) for k in range(3))<=1)
model.addConstr(gb.quicksum(x[13,j,k] for j in range(3) for k in range(3))<=1)
model.addConstr(gb.quicksum(x[15,j,k] for j in range(3) for k in range(3))<=1)
model.addConstr(gb.quicksum(x[17,j,k] for j in range(3) for k in range(3))<=1)
model.addConstr(gb.quicksum(x[18,j,k] for j in range(3) for k in range(3))<=1)
#set RHS to 0 for adjusted constraint for no reserve physicians should work


<gurobi.Constr *Awaiting Model Update*>

Physician One Should Work at Least Once on Wednesday

In [15]:
model.addConstr(gb.quicksum(x[0,j,2] for j in range(3))>=1)

<gurobi.Constr *Awaiting Model Update*>

Optimize Model and Print 

In [16]:
# Optimally solve the problem
model.optimize()

# Print the objective and decision variables
model.printAttr('X')

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 231 rows, 252 columns and 730 nonzeros
Model fingerprint: 0x0cf3029e
Variable types: 0 continuous, 252 integer (252 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [5e+04, 5e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+03]
Presolve removed 133 rows and 74 columns
Presolve time: 0.00s
Presolved: 98 rows, 178 columns, 418 nonzeros
Variable types: 0 continuous, 178 integer (178 binary)
Found heuristic solution: objective 1.243494e+07

Root relaxation: objective 1.084608e+07, 36 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1.0846e+07    0    9 1.2435e+07 1.084